#  Making masks for samples based on XRF signal.

In [5]:
#Import necessary libaries

%matplotlib widget
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import h5py
import scipy.ndimage as ndimage
#To import DanMAX from the folder above:
sys.path.append('../')
import DanMAX as DM
from lib.xrf_pyMCA_fit import xrfBatch
style = DM.darkMode(style_dic={'figure.figsize':'large'})

## Define the samples and all the groups

could also be ecpanded with more information about each sample

In [6]:
groups = DM.getProposalScans()

In [28]:
#Define the location where the XRF fits are saved within the XRF file
xrf_h5_fit_path = 'xrf_fits/xrf_fit/results/parameters/Ca_K'
proposal, visit = DM.getCurrentProposal()
bin_struct = ndimage.iterate_structure(ndimage.generate_binary_structure(2,1),1)
for group in groups:
    new_file = True
    for sample in groups[group]:
        scans = groups[group][sample]
        xrf_fit_dir, xrf_fit_file = DM.mapping.getXRFFitFilename(scans,proposal=proposal,visit=visit)
        xrf_fits_filename = f'{xrf_fit_dir}/elements/{xrf_fit_file}.h5' 
        mask_foldername =  '/'.join(xrf_fit_dir.split('/')[0:-3]).replace('xrf_fit','mask')

        if not os.path.isdir(mask_foldername):
            os.makedirs(mask_foldername)
            os.chmod(mask_foldername, 0o770)

        
        with h5py.File(xrf_fits_filename, 'r' ) as xrf_file:
            ca = xrf_file[xrf_h5_fit_path][:]
        binary_image = ndimage.binary_erosion(ndimage.binary_dilation(ca>200,bin_struct),bin_struct)
        labels, n_features = ndimage.label(binary_image)
        size = ndimage.sum(binary_image,labels,range(n_features+1))
        
        mask = size[labels] > 300
        mask_file = os.path.join(mask_foldername,f'{group}.h5')
        if new_file:
            new_file = False
            with h5py.File(mask_file,'w') as mf:
                mf.create_dataset(f'mask/{sample}', data = mask)
        else:
            with h5py.File(mask_file,'a') as mf:
                mf.create_dataset(f'mask/{sample}', data = mask)

(3, 3)
